In [5]:
import sys
#from classifier import cleaned_data as data
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
DEBUG = False
if '-v' in sys.argv:
  DEBUG = True

def log(message):
  if DEBUG:
    print(message)


"""Need to figure out what the best format of the data is"""

df = pd.read_csv("crowd.tsv", encoding = "iso-8859-1", sep = '\t')
df = df[['id', 'question', 'opinion']]
df

,id,question,opinion
0,0,Who invented the light bulb?,2
1,0,Who is the only woman to have ever received tw...,2
2,0,What's the most expensive material on Earth?,2
3,0,What's the name of the final of the NFL (Natio...,2
4,0,What's the second country with more Olympic me...,1
5,0,How many modern Olympic Summer Games have ther...,1
6,0,Who lives in a pineapple under the sea?,3
7,0,What's the name of the reindeer in Frozen?,3
8,0,What is the top color in a rainbow?,3
9,0,For which song did Toy Story get nominated to ...,3


In [15]:
mean = df.groupby(['id'], as_index = False, sort = False).mean().rename(columns = {'opinion' : 'opinion_mean'})[['id', 'opinion_mean']]
Opinions = pd.merge(df, mean, on = 'id', how = 'left', sort = False)
Opinions['adjusted'] = Opinions['opinion'] - Opinions['opinion_mean']
Opinions

,id,question,opinion,opinion_mean,adjusted
0,0,Who invented the light bulb?,2,2.533333,-0.533333
1,0,Who is the only woman to have ever received tw...,2,2.533333,-0.533333
2,0,What's the most expensive material on Earth?,2,2.533333,-0.533333
3,0,What's the name of the final of the NFL (Natio...,2,2.533333,-0.533333
4,0,What's the second country with more Olympic me...,1,2.533333,-1.533333
5,0,How many modern Olympic Summer Games have ther...,1,2.533333,-1.533333
6,0,Who lives in a pineapple under the sea?,3,2.533333,0.466667
7,0,What's the name of the reindeer in Frozen?,3,2.533333,0.466667
8,0,What is the top color in a rainbow?,3,2.533333,0.466667
9,0,For which song did Toy Story get nominated to ...,3,2.533333,0.466667


In [19]:
result = pd.DataFrame({'id':Opinions['id'], "question": Opinions['question'], "opinion" : Opinions['adjusted']})
result1 = result.pivot_table(index = 'id', columns = 'question', values = 'opinion').fillna(0)
result1

question,'El ClÃÂ¡sico' is a game between which club giants?,A type glass that is highly resistant to heat?,"How many players, including the goaltender, make up an ice hockey team?",7 rings' is a song by which American singer?,A device used to measure the strength of magnetic field.,A winner in Boxing (Light Flyweight) at the 2012 Summer Olympics? He won a gold medal in the 2008 Summer Olympics as well.,"According to esportsearnings.com 2019 what is the top prize money awarding game of all time? - ""League of Legends""? ""World of Warcraft""? ""Dota 2""?","According to the ABRSM top list, what is the most popular instrument to learn by 2014? - ""Guitar""? ""Piano""? ""Keyboard""?","According to xygaming.com what country is the highest earning contry by esports prize money? - ""South Korea""? ""China""? ""Denmark""?","An ordinary bit can have two states. How many can a quantum bit, or qubit, have?",...,"Who won the world championship in handball 2019? - ""France""? ""Denmark""? ""Germany""? ""Norway""?",Who won the world cup in soccer in 2018?,Who wrote the 1976 hit 'Rich Girl'?,Who wrote the little match girl?,Who wrote the play 'Hamlet'?,"Who wrote the song famous for the lyrics: ""Never gonna give you up, never gonna let you down.""",With who was Nemo's father accompanied during looking for his son?,WoW is an acronym for which Blizzard game?,what colour is the sky?,which year the Minecrafts was released?
id,,,,,,,,,,,,,,,,,,,,,
0,0.000000,-1.533333,0.000000,0.466667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.466667,0.000000,0.000000
1,0.000000,-0.022222,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.977778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.311111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,-1.266667,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.266667
5,0.688889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.688889,...,0.000000,0.000000,0.000000,0.000000,0.688889,-1.311111,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,-1.422222,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [47]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(result1)
resulty = np.argsort(cosine_similarity(result1))
user = 10
k = 4
np.flip(resulty[10, -(k+1): 37])

array([23, 13, 12,  8])

array([[ 1.00000000e+00, -4.13971947e-02,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00, -7.27149212e-04],
       [-4.13971947e-02,  1.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  2.77918944e-04],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  1.00000000e+00, -2.83295051e-04],
       [-7.27149212e-04,  0.00000000e+00,  2.77918944e-04, ...,
         0.00000000e+00, -2.83295051e-04,  1.00000000e+00]])